In [2]:
from pandas import read_csv
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import grangercausalitytests
from numpy import log
import numpy as np
import pandas as pd
import datetime as datetime
import csv

def zscore(df, col):
    col_mean = df[col].rolling(window=14, center=True).mean()
    col_std = df[col].rolling(window=14, center=True).std()
    return (df[col] - col_mean)/col_std

df = pd.read_csv('./data/11-26/daily_plutchik_threshold_5.csv', parse_dates=['month/day'])
approval_rating_df = pd.read_csv('./data/approval_polllist.csv', parse_dates=["enddate"])
topic_df = pd.read_excel('./data/TopicScoresPerDay_Final.xlsx')
sentiment_df = pd.read_csv('./data/12-6/daily_senti_corrected.csv', parse_dates=['month/day'])

In [3]:
df["Anticipation Z-Score"] = zscore(df, "Anticipation")
df["Trust Z-Score"] = zscore(df, "Trust")
df["Surprise Z-Score"] = zscore(df, "Surprise")
df["Sadness Z-Score"] = zscore(df, "Sadness")
df["Joy Z-Score"] = zscore(df, "Joy")
df["Fear Z-Score"] = zscore(df, "Fear")
df["Disgust Z-Score"] = zscore(df, "Disgust")
df["Anger Z-Score"] = zscore(df, "Anger")

start_date = datetime.datetime(2017, 7, 12, 0, 0)
end_date = datetime.datetime(2017, 12, 31, 0, 0)

dated_emotion_df = df[(df["month/day"] > start_date) & (df["month/day"] < end_date)]

In [4]:
topic_df = topic_df.drop(['Filename', 'Segment', 'WC', 'WPS', 'Sixltr', 'Dic', 
                          'AllPunc', 'Period', 'Comma', 'Colon', 'SemiC', 'QMark', 
                          'Exclam', 'Dash', 'Quote', 'Apostro', 'Parenth', 'OtherP', 
                          'Em1', 'Em2', 'Em3', 'Em4', 'Em5', 'Em6', 'Em7', 'Em8', 'Gallup Value'], axis=1)

for (columnName, columnData) in topic_df.iteritems():
    topic_df[columnName + " Z-Score"] = zscore(topic_df, columnName)
    
topic_df = topic_df[66:237]
topic_df.fillna(0, inplace=True)
topic_df.head()

,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,...,topic492 Z-Score,topic493 Z-Score,topic494 Z-Score,topic495 Z-Score,topic496 Z-Score,topic497 Z-Score,topic498 Z-Score,topic499 Z-Score,topic500 Z-Score,topic501 Z-Score
66,6.46,0.0,0.00,0.05,0.0,0.01,0.64,0.57,0.19,0.65,...,0.194023,2.163111,1.089956,-0.491032,1.429682,0.413594,-0.702665,1.065521,1.432840,-0.507075
67,6.12,0.0,0.00,0.04,0.0,0.01,0.46,0.65,0.08,0.49,...,0.769534,-0.151553,-0.433117,-0.479883,0.629919,1.120638,-0.401764,-1.296174,-1.321625,-0.372797
68,6.20,0.0,0.01,0.01,0.0,0.02,0.61,0.51,0.10,0.69,...,0.461597,-1.572928,-0.890207,1.424433,1.097163,1.309574,-0.475466,-1.355222,-2.163202,-0.066748
69,6.08,0.0,0.00,0.03,0.0,0.02,0.47,0.50,0.07,0.67,...,-0.503236,0.630641,2.372378,2.064031,-0.495156,-0.901418,-0.321563,0.105594,0.360875,-0.514077
70,5.97,0.0,0.00,0.06,0.0,0.02,0.45,0.63,0.06,0.21,...,-0.493518,-0.165357,-1.091842,-0.820724,-0.338300,-1.210769,-1.583182,1.308417,0.067327,-0.723097


In [5]:
for (columnName, columnData) in sentiment_df.iteritems():
    if columnName != "month/day":
        sentiment_df[columnName+" Z-Score"] = zscore(sentiment_df, columnName)
sentiment_df = sentiment_df[(sentiment_df["month/day"] > start_date) & (sentiment_df["month/day"] < end_date)]
sentiment_df.head()

,month/day,pos_sentiment,neg_sentiment,neu_sentiment,pos_sentiment Z-Score,neg_sentiment Z-Score,neu_sentiment Z-Score
66,2017-07-13,0.418653,0.391710,0.189637,1.339725,-0.882616,-0.535477
67,2017-07-14,0.389543,0.390280,0.220177,0.723188,-0.960567,0.695292
68,2017-07-15,0.412959,0.376269,0.210773,1.445375,-1.252709,0.201804
69,2017-07-16,0.340331,0.465886,0.193783,-0.453674,0.663924,-0.590644
70,2017-07-17,0.357298,0.470588,0.172113,0.058015,0.762438,-1.515081


In [6]:
def check_stationarity(column, title):
    result = adfuller(column)
    for key, value in result[4].items():
        if result[0] > value:
            print('Found a non-stationary time series:', title)
            print('ADF Statistic: %f' % result[0])
            print('p-value: %f' % result[1])
            for key, value in result[4].items():
                print('\t%s: %.3f' % (key, value))
        if result[1] > 0.05:
            print('Found a p-value greater than threshold')

In [7]:
for (columnName, columnData) in sentiment_df.iteritems():
    if columnName.endswith("Z-Score"):
        check_stationarity(sentiment_df[columnName], columnName)
        
sentiment_df.head()

,month/day,pos_sentiment,neg_sentiment,neu_sentiment,pos_sentiment Z-Score,neg_sentiment Z-Score,neu_sentiment Z-Score
66,2017-07-13,0.418653,0.391710,0.189637,1.339725,-0.882616,-0.535477
67,2017-07-14,0.389543,0.390280,0.220177,0.723188,-0.960567,0.695292
68,2017-07-15,0.412959,0.376269,0.210773,1.445375,-1.252709,0.201804
69,2017-07-16,0.340331,0.465886,0.193783,-0.453674,0.663924,-0.590644
70,2017-07-17,0.357298,0.470588,0.172113,0.058015,0.762438,-1.515081


In [8]:
for i in range(1, 502):
    title = 'topic'+str(i)+' Z-Score'
    check_stationarity(topic_df[title], title)

/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/regression/linear_model.py:867: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1294: RuntimeWarning: invalid value encountered in true_divide
  return self.params / self.bse


In [9]:
for (columnName, columnData) in dated_emotion_df.iteritems():
    if columnName.endswith('Z-Score'):
        check_stationarity(dated_emotion_df[columnName], columnName)

# p-values are all <= 0.05 and ADF Statistics are < values at 1%, 5%, and 10%

In [10]:
truncated_approvals = approval_rating_df.loc[approval_rating_df['pollster'] == 'Ipsos']
truncated_approvals = truncated_approvals.loc[truncated_approvals['subgroup'] == 'All polls']
truncated_approvals = truncated_approvals[(truncated_approvals['enddate'] > start_date) & (truncated_approvals['enddate'] < end_date)]
adjusted_and_unadjusted_approvals = truncated_approvals[['adjusted_approve', 'approve']]
truncated_approvals = truncated_approvals['adjusted_approve']

approvals_array = truncated_approvals.values

In [11]:
to_merge_sentiment = sentiment_df[['pos_sentiment', 'neg_sentiment', 'neu_sentiment', 'pos_sentiment Z-Score', 'neg_sentiment Z-Score', 'neu_sentiment Z-Score']]
to_merge_sentiment = to_merge_sentiment.reset_index(drop=True)
to_merge_emotion = dated_emotion_df.drop('month/day', axis=1)
to_merge_emotion = to_merge_emotion.reset_index(drop=True)
to_merge_approvals = adjusted_and_unadjusted_approvals.reset_index(drop=True)
to_merge_topics = topic_df.reset_index(drop=True)
all_data_to_upload = pd.concat([to_merge_sentiment, to_merge_emotion, to_merge_topics, to_merge_approvals], axis=1)
print(len(to_merge_sentiment), len(dated_emotion_df), len(topic_df), len(truncated_approvals))
all_data_to_upload = all_data_to_upload.rename(columns={'adjusted_approve': 'Ipsos Adjusted Approval', 'approve': 'Ipsos Approval'})
all_data_to_upload.to_csv("12-6_all_data.csv")
all_data_to_upload.head()

171 171 171 171


,pos_sentiment,neg_sentiment,neu_sentiment,pos_sentiment Z-Score,neg_sentiment Z-Score,neu_sentiment Z-Score,Anger,Disgust,Fear,Joy,...,topic494 Z-Score,topic495 Z-Score,topic496 Z-Score,topic497 Z-Score,topic498 Z-Score,topic499 Z-Score,topic500 Z-Score,topic501 Z-Score,Ipsos Adjusted Approval,Ipsos Approval
0,0.418653,0.391710,0.189637,1.339725,-0.882616,-0.535477,0.025688,0.022018,0.036697,0.122936,...,1.089956,-0.491032,1.429682,0.413594,-0.702665,1.065521,1.432840,-0.507075,41.35173,40.6
1,0.389543,0.390280,0.220177,0.723188,-0.960567,0.695292,0.015248,0.017789,0.054638,0.121982,...,-0.433117,-0.479883,0.629919,1.120638,-0.401764,-1.296174,-1.321625,-0.372797,42.25173,41.5
2,0.412959,0.376269,0.210773,1.445375,-1.252709,0.201804,0.013717,0.019204,0.046639,0.130316,...,-0.890207,1.424433,1.097163,1.309574,-0.475466,-1.355222,-2.163202,-0.066748,40.95173,40.2
3,0.340331,0.465886,0.193783,-0.453674,0.663924,-0.590644,0.022744,0.030814,0.038151,0.118855,...,2.372378,2.064031,-0.495156,-0.901418,-0.321563,0.105594,0.360875,-0.514077,40.95173,40.2
4,0.357298,0.470588,0.172113,0.058015,0.762438,-1.515081,0.022989,0.007663,0.034483,0.099617,...,-1.091842,-0.820724,-0.338300,-1.210769,-1.583182,1.308417,0.067327,-0.723097,38.45173,37.7


In [12]:
def find_significant_time_series(time_series, columnName, result_set):
    x = np.asarray([approvals_array, time_series[columnName]]).T
    results = grangercausalitytests(x, maxlag=7, verbose=False)
    for lag, test in results.items():
        for test, values in results[lag][0].items():
            if values[1] <= 0.05:
                print("found a significant test result for column:", columnName, "with", lag, "lags")
                result_set.add(columnName)

In [13]:
significant_emotions = set()
for (columnName, columnData) in dated_emotion_df.iteritems():
    if columnName.endswith('Z-Score'):
        find_significant_time_series(dated_emotion_df, columnName, significant_emotions)

found a significant test result for column: Trust Z-Score with 1 lags
found a significant test result for column: Trust Z-Score with 1 lags
found a significant test result for column: Trust Z-Score with 1 lags
found a significant test result for column: Trust Z-Score with 1 lags
found a significant test result for column: Trust Z-Score with 2 lags
found a significant test result for column: Trust Z-Score with 2 lags
found a significant test result for column: Trust Z-Score with 2 lags
found a significant test result for column: Trust Z-Score with 2 lags
found a significant test result for column: Trust Z-Score with 4 lags
found a significant test result for column: Trust Z-Score with 4 lags
found a significant test result for column: Trust Z-Score with 4 lags
found a significant test result for column: Trust Z-Score with 4 lags
found a significant test result for column: Trust Z-Score with 5 lags
found a significant test result for column: Trust Z-Score with 5 lags
found a significant 

In [14]:
for emotion in significant_emotions:
    print(emotion)

Joy Z-Score
Surprise Z-Score
Trust Z-Score


In [21]:
significant_sentiments = set()
for (columnName, columnData) in sentiment_df.iteritems():
    if columnName.endswith("Z-Score"):
        find_significant_time_series(sentiment_df, columnName, significant_sentiments)

found a significant test result for column: pos_sentiment Z-Score with 6 lags
found a significant test result for column: pos_sentiment Z-Score with 6 lags
found a significant test result for column: pos_sentiment Z-Score with 6 lags
found a significant test result for column: pos_sentiment Z-Score with 6 lags
found a significant test result for column: pos_sentiment Z-Score with 7 lags
found a significant test result for column: pos_sentiment Z-Score with 7 lags
found a significant test result for column: neg_sentiment Z-Score with 6 lags
found a significant test result for column: neg_sentiment Z-Score with 6 lags
found a significant test result for column: neg_sentiment Z-Score with 6 lags
found a significant test result for column: neg_sentiment Z-Score with 6 lags
found a significant test result for column: neg_sentiment Z-Score with 7 lags
found a significant test result for column: neg_sentiment Z-Score with 7 lags
found a significant test result for column: neg_sentiment Z-Scor

In [22]:
for sentiment in significant_sentiments:
    print(sentiment)

pos_sentiment Z-Score
neg_sentiment Z-Score


In [15]:
significant_topics = set()
for (columnName, columnData) in topic_df.iteritems():
    if columnName.endswith('Z-Score'):
        find_significant_time_series(topic_df, columnName, significant_topics)

/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1768: RuntimeWarning: invalid value encountered in true_divide
  F /= J
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 2, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 3, but rank is 0
  'rank is %d' % (J, J_), ValueWarning)
/Users/suvir/anaconda3/lib/python3.6/site-packages/statsmodels/base/model.py:1752: ValueWarning: covariance of constraints does not have full rank. T

found a significant test result for column: topic6 Z-Score with 1 lags
found a significant test result for column: topic6 Z-Score with 1 lags
found a significant test result for column: topic6 Z-Score with 1 lags
found a significant test result for column: topic6 Z-Score with 1 lags
found a significant test result for column: topic6 Z-Score with 2 lags
found a significant test result for column: topic6 Z-Score with 2 lags
found a significant test result for column: topic6 Z-Score with 2 lags
found a significant test result for column: topic6 Z-Score with 2 lags
found a significant test result for column: topic6 Z-Score with 3 lags
found a significant test result for column: topic6 Z-Score with 3 lags
found a significant test result for column: topic6 Z-Score with 3 lags
found a significant test result for column: topic6 Z-Score with 3 lags
found a significant test result for column: topic10 Z-Score with 2 lags
found a significant test result for column: topic28 Z-Score with 1 lags
foun

found a significant test result for column: topic79 Z-Score with 1 lags
found a significant test result for column: topic79 Z-Score with 1 lags
found a significant test result for column: topic79 Z-Score with 1 lags
found a significant test result for column: topic79 Z-Score with 1 lags
found a significant test result for column: topic79 Z-Score with 2 lags
found a significant test result for column: topic79 Z-Score with 2 lags
found a significant test result for column: topic79 Z-Score with 2 lags
found a significant test result for column: topic79 Z-Score with 2 lags
found a significant test result for column: topic79 Z-Score with 5 lags
found a significant test result for column: topic79 Z-Score with 5 lags
found a significant test result for column: topic81 Z-Score with 6 lags
found a significant test result for column: topic81 Z-Score with 6 lags
found a significant test result for column: topic81 Z-Score with 7 lags
found a significant test result for column: topic88 Z-Score with

found a significant test result for column: topic132 Z-Score with 1 lags
found a significant test result for column: topic132 Z-Score with 1 lags
found a significant test result for column: topic132 Z-Score with 1 lags
found a significant test result for column: topic132 Z-Score with 1 lags
found a significant test result for column: topic132 Z-Score with 2 lags
found a significant test result for column: topic132 Z-Score with 2 lags
found a significant test result for column: topic132 Z-Score with 2 lags
found a significant test result for column: topic132 Z-Score with 2 lags
found a significant test result for column: topic132 Z-Score with 4 lags
found a significant test result for column: topic132 Z-Score with 4 lags
found a significant test result for column: topic132 Z-Score with 4 lags
found a significant test result for column: topic132 Z-Score with 4 lags
found a significant test result for column: topic132 Z-Score with 5 lags
found a significant test result for column: topic13

found a significant test result for column: topic167 Z-Score with 4 lags
found a significant test result for column: topic167 Z-Score with 4 lags
found a significant test result for column: topic167 Z-Score with 4 lags
found a significant test result for column: topic167 Z-Score with 4 lags
found a significant test result for column: topic167 Z-Score with 5 lags
found a significant test result for column: topic167 Z-Score with 5 lags
found a significant test result for column: topic167 Z-Score with 5 lags
found a significant test result for column: topic167 Z-Score with 5 lags
found a significant test result for column: topic167 Z-Score with 6 lags
found a significant test result for column: topic167 Z-Score with 6 lags
found a significant test result for column: topic174 Z-Score with 5 lags
found a significant test result for column: topic174 Z-Score with 5 lags
found a significant test result for column: topic174 Z-Score with 5 lags
found a significant test result for column: topic17

...

found a significant test result for column: topic265 Z-Score with 6 lags
found a significant test result for column: topic265 Z-Score with 6 lags
found a significant test result for column: topic265 Z-Score with 6 lags
found a significant test result for column: topic265 Z-Score with 6 lags
found a significant test result for column: topic266 Z-Score with 4 lags
found a significant test result for column: topic266 Z-Score with 4 lags
found a significant test result for column: topic266 Z-Score with 4 lags
found a significant test result for column: topic266 Z-Score with 4 lags
found a significant test result for column: topic266 Z-Score with 5 lags
found a significant test result for column: topic266 Z-Score with 5 lags
found a significant test result for column: topic266 Z-Score with 5 lags
found a significant test result for column: topic266 Z-Score with 5 lags
found a significant test result for column: topic266 Z-Score with 6 lags
found a significant test result for column: topic27

found a significant test result for column: topic333 Z-Score with 1 lags
found a significant test result for column: topic333 Z-Score with 1 lags
found a significant test result for column: topic333 Z-Score with 1 lags
found a significant test result for column: topic333 Z-Score with 1 lags
found a significant test result for column: topic333 Z-Score with 2 lags
found a significant test result for column: topic333 Z-Score with 2 lags
found a significant test result for column: topic333 Z-Score with 2 lags
found a significant test result for column: topic333 Z-Score with 2 lags
found a significant test result for column: topic333 Z-Score with 3 lags
found a significant test result for column: topic333 Z-Score with 3 lags
found a significant test result for column: topic333 Z-Score with 3 lags
found a significant test result for column: topic333 Z-Score with 3 lags
found a significant test result for column: topic333 Z-Score with 4 lags
found a significant test result for column: topic33

found a significant test result for column: topic369 Z-Score with 5 lags
found a significant test result for column: topic369 Z-Score with 6 lags
found a significant test result for column: topic369 Z-Score with 6 lags
found a significant test result for column: topic369 Z-Score with 6 lags
found a significant test result for column: topic369 Z-Score with 6 lags
found a significant test result for column: topic374 Z-Score with 1 lags
found a significant test result for column: topic376 Z-Score with 3 lags
found a significant test result for column: topic376 Z-Score with 3 lags
found a significant test result for column: topic376 Z-Score with 3 lags
found a significant test result for column: topic376 Z-Score with 3 lags
found a significant test result for column: topic377 Z-Score with 1 lags
found a significant test result for column: topic377 Z-Score with 1 lags
found a significant test result for column: topic377 Z-Score with 1 lags
found a significant test result for column: topic37

found a significant test result for column: topic444 Z-Score with 2 lags
found a significant test result for column: topic444 Z-Score with 2 lags
found a significant test result for column: topic444 Z-Score with 2 lags
found a significant test result for column: topic444 Z-Score with 2 lags
found a significant test result for column: topic448 Z-Score with 2 lags
found a significant test result for column: topic448 Z-Score with 2 lags
found a significant test result for column: topic448 Z-Score with 2 lags
found a significant test result for column: topic448 Z-Score with 2 lags
found a significant test result for column: topic448 Z-Score with 3 lags
found a significant test result for column: topic448 Z-Score with 3 lags
found a significant test result for column: topic448 Z-Score with 3 lags
found a significant test result for column: topic448 Z-Score with 3 lags
found a significant test result for column: topic448 Z-Score with 4 lags
found a significant test result for column: topic44

found a significant test result for column: topic489 Z-Score with 1 lags
found a significant test result for column: topic489 Z-Score with 1 lags
found a significant test result for column: topic489 Z-Score with 1 lags
found a significant test result for column: topic489 Z-Score with 1 lags
found a significant test result for column: topic489 Z-Score with 2 lags
found a significant test result for column: topic489 Z-Score with 2 lags
found a significant test result for column: topic489 Z-Score with 4 lags
found a significant test result for column: topic489 Z-Score with 4 lags
found a significant test result for column: topic489 Z-Score with 5 lags
found a significant test result for column: topic489 Z-Score with 5 lags
found a significant test result for column: topic489 Z-Score with 5 lags
found a significant test result for column: topic489 Z-Score with 5 lags
found a significant test result for column: topic491 Z-Score with 3 lags
found a significant test result for column: topic49

In [20]:
print(len(significant_topics))
for topic in significant_topics:
    print(topic)
    
with (open("significant_topics.csv",'w')) as file:
    writer = csv.writer(file)
    for topic in significant_topics:
        writer.writerow([topic])

124
topic448 Z-Score
topic492 Z-Score
topic337 Z-Score
topic374 Z-Score
topic473 Z-Score
topic478 Z-Score
topic104 Z-Score
topic10 Z-Score
topic289 Z-Score
topic386 Z-Score
topic142 Z-Score
topic98 Z-Score
topic399 Z-Score
topic461 Z-Score
topic162 Z-Score
topic231 Z-Score
topic203 Z-Score
topic157 Z-Score
topic380 Z-Score
topic404 Z-Score
topic452 Z-Score
topic382 Z-Score
topic333 Z-Score
topic346 Z-Score
topic319 Z-Score
topic148 Z-Score
topic40 Z-Score
topic79 Z-Score
topic491 Z-Score
topic115 Z-Score
topic262 Z-Score
topic112 Z-Score
topic377 Z-Score
topic366 Z-Score
topic218 Z-Score
topic369 Z-Score
topic70 Z-Score
topic237 Z-Score
topic117 Z-Score
topic105 Z-Score
topic6 Z-Score
topic265 Z-Score
topic321 Z-Score
topic126 Z-Score
topic39 Z-Score
topic211 Z-Score
topic482 Z-Score
topic221 Z-Score
topic301 Z-Score
topic99 Z-Score
topic351 Z-Score
topic274 Z-Score
topic434 Z-Score
topic239 Z-Score
topic430 Z-Score
topic444 Z-Score
topic200 Z-Score
topic303 Z-Score
topic255 Z-Score
to

# Example of Granger Causality Test output 

In [17]:
x = np.asarray([approvals_array, dated_emotion_df["Trust Z-Score"]]).T
grangercausalitytests(x, maxlag=7)


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=5.4970  , p=0.0202  , df_denom=167, df_num=1
ssr based chi2 test:   chi2=5.5957  , p=0.0180  , df=1
likelihood ratio test: chi2=5.5056  , p=0.0190  , df=1
parameter F test:         F=5.4970  , p=0.0202  , df_denom=167, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=3.4721  , p=0.0334  , df_denom=164, df_num=2
ssr based chi2 test:   chi2=7.1558  , p=0.0279  , df=2
likelihood ratio test: chi2=7.0085  , p=0.0301  , df=2
parameter F test:         F=3.4721  , p=0.0334  , df_denom=164, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=2.2669  , p=0.0828  , df_denom=161, df_num=3
ssr based chi2 test:   chi2=7.0963  , p=0.0689  , df=3
likelihood ratio test: chi2=6.9506  , p=0.0735  , df=3
parameter F test:         F=2.2669  , p=0.0828  , df_denom=161, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=2.8883  , p=0.0242  

{1: ({'lrtest': (5.505609732254527, 0.018955570283706202, 1),
   'params_ftest': (5.4969839581966466, 0.020225482168537815, 167.0, 1.0),
   'ssr_chi2test': (5.5957321730144933, 0.018004284631946768, 1),
   'ssr_ftest': (5.4969839581965916, 0.020225482168538346, 167.0, 1)},
   array([[ 0.,  1.,  0.]])]),
 2: ({'lrtest': (7.0084671586536729, 0.030069810639427971, 2),
   'params_ftest': (3.4720538653599715, 0.03335465188161163, 164.0, 2.0),
   'ssr_chi2test': (7.1558183322656976, 0.027934042682226719, 2),
   'ssr_ftest': (3.4720538653596877, 0.033354651881620734, 164.0, 2)},
   array([[ 0.,  0.,  1.,  0.,  0.],
          [ 0.,  0.,  0.,  1.,  0.]])]),
 3: ({'lrtest': (6.9505606174623153, 0.073490379870249353, 3),
   'params_ftest': (2.266887915829042, 0.082772383454681672, 161.0, 3.0),
   'ssr_chi2test': (7.0963447799867563, 0.068889518125820434, 3),
   'ssr_ftest': (2.2668879158291024, 0.082772383454676163, 161.0, 3)},
   array([[ 0.,  0.,  0.,  1.,  0.,  0.,  0.],
          [ 0.,  0.,  